In [101]:
import sys
import time
import numpy as np
import random
import time
import datetime

from numba import cuda, float32
from numba import jit
from numba import vectorize

from numpy import arange



#### создаем две матрицы одного размера

In [167]:

M = 10  # количество строк и столбцов в матрицах


start_time = datetime.datetime.now() # отметили стартовое время

z1 = []
z2 = []

for x in range (M):
    for y in range (M):
        z1.append(float(random.randint(0., 9.)))  # заполняем два массива размером М*М числами от 1 до 9
        z2.append(float(random.randint(0., 9.)))
        A = np.array(z1)
        B = np.array(z2)

A = A.reshape(M, M) #преобразем массивы в матрицы
B = B.reshape(M, M)

C = np.zeros((M, M))

print ('time:  '+ str(datetime.datetime.now() - start_time)) # из стартового времени вычитаем текущее

time:  0:00:00.006000


In [159]:
#print(A)


In [160]:
#print(B)

In [161]:
#print(C)

In [162]:
#print(type(C[0][0]), type(A[0][0]))

#### Умножение матриц с использованием служебной функции

In [163]:
start_time = datetime.datetime.now() # отметили стартовое время

C1 = A.dot(B)


print(C1)
print ('time:  '+ str(datetime.datetime.now() - start_time)) # из стартового времени вычитаем текущее

[[301. 210. 195. 274. 177. 262. 183. 232.  79. 159.]
 [118. 124.  65. 143.  67. 172.  96. 153.  80. 147.]
 [189. 177.  85. 201.  90. 173.  97. 193. 102. 151.]
 [241. 249. 157. 252. 120. 245. 132. 279. 143. 183.]
 [347. 281. 160. 344. 165. 348. 201. 325. 205. 259.]
 [309. 250. 189. 318. 169. 310. 195. 291. 146. 194.]
 [187. 195. 113. 176. 110. 193. 138. 178.  73. 155.]
 [420. 332. 239. 391. 233. 408. 273. 360. 173. 308.]
 [267. 193. 149. 268. 138. 249. 145. 237. 121. 156.]
 [373. 338. 238. 353. 195. 400. 269. 358. 211. 268.]]
time:  0:00:00


#### Функция для умножения матриц (на CPU)

In [164]:
def mat_mul (matr1, matr2, matr3):
    
    matr3 = np.zeros((M, M)) # сознаем матрицу размером МхМ из нулей
    for i in range(0, M):
        for j in range(0, M):
            for k in range(0, M):
                matr3[i][j] += matr1[i][k] * matr2[k][j]  # по формуле умножения матриц
    
    print (matr3)
    
start_time = datetime.datetime.now() # отметили стартовое время   

mat_mul(A, B, C)

print ('time:  '+ str(datetime.datetime.now() - start_time)) # из стартового времени вычитаем текущее    


[[301. 210. 195. 274. 177. 262. 183. 232.  79. 159.]
 [118. 124.  65. 143.  67. 172.  96. 153.  80. 147.]
 [189. 177.  85. 201.  90. 173.  97. 193. 102. 151.]
 [241. 249. 157. 252. 120. 245. 132. 279. 143. 183.]
 [347. 281. 160. 344. 165. 348. 201. 325. 205. 259.]
 [309. 250. 189. 318. 169. 310. 195. 291. 146. 194.]
 [187. 195. 113. 176. 110. 193. 138. 178.  73. 155.]
 [420. 332. 239. 391. 233. 408. 273. 360. 173. 308.]
 [267. 193. 149. 268. 138. 249. 145. 237. 121. 156.]
 [373. 338. 238. 353. 195. 400. 269. 358. 211. 268.]]
time:  0:00:00.018601


#### Результат работы на GPU с применением CUDA

In [185]:
@cuda.jit
def mat_mul_GPU(matr1, matr2, matr3):
    
    i, j = cuda.grid(2)
    if i < matr3.shape[0] and j < matr3.shape[1]:
        for k in range(matr1.shape[1]):
            matr3[i][j] += matr1[i][k] * matr2[k][j]
    return matr3
        


def hosts(matr1, matr2, matr3):   
    
    
    d_matr1 = cuda.to_device(matr1) # выделили память
    d_matr2 = cuda.to_device(matr2)
    d_matr3 = cuda.device_array(d_matr3.shape, np.float32)
    
    block = (TPB, TPB) # размер блока
    
    blockgrid_x = math.ceil(matr1.shape[0]/ block[0]) # строки первой матрицы
    blockgrid_y = math.ceil(matr1.shape[1]/ block[1]) # столбцы второй матрицы
    blockgrid = (blockgrid_x, blockgrid_y)
    
    
    mat_mul_GPU[blockgrid, block](d_matr1, d_matr2, d_matr3)
    
    return d_matr3.copy_to_host()

In [186]:
TPB = 2 # потоков
start_time = datetime.datetime.now() # отметили стартовое время   

hosts(A, B, C)

print ('time:  '+ str(datetime.datetime.now() - start_time)) # из стартового времени вычитаем текущее    

CudaSupportError: <CUDA device 0 'b'GeForce 9600 GSO 512''> has compute capability < (2, 0)